In [1]:
# !pip install -r requirements.txt

In [2]:
# !wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml<br>
# !sudo docker-compose up -d

In [2]:
from pymilvus import connections
import psycopg2

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from flask import Flask, request
from flask_cors import CORS, cross_origin

from OpenSSL import SSL

/home/milan_v/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## Database Connections

In [83]:
connections.connect(host='104.196.170.44', port='19530')
conn = psycopg2.connect(database="railway", host='containers-us-west-100.railway.app', port='5754', user='postgres', password='5nli3lsLxl0XoBLU4syO') # postgres
print("CONNECTION ====",conn)

CONNECTION ==== <connection object at 0x7f3e7069ce00; dsn: 'user=postgres password=xxx dbname=railway host=containers-us-west-100.railway.app port=5754', closed: 0>


## Global Variables

In [84]:
TABLE_NAME = "Skills_Embd"
field_name = "Skills"

TABLE_NAME_2 = "Embds"
field_name_2 = "Mix_Fields"

TABLE_NAME_3 = "Biz_Embds"
field_name_3 = "Biz"

TABLE_NAME_4 = "Ppl_Embds"
field_name_4 = "Ppl"

TABLE_NAME_5 = "cmt_Embds"
field_name_5 = "cmt"

# paraphrase-mpnet-base-v2 
# multi-qa-mpnet-base-dot-v1
# all-with_prefix-t5-base-v1
# all-mpnet-base-v2
# all-MiniLM-L6-v2
# paraphrase-MiniLM-L6-v2 **BEST**
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
cursor = conn.cursor()

search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

## Column Names

In [34]:
# cursor = conn.cursor()
# cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
# print(cursor.fetchall())

[('hypertable',), ('hypertable_data_node',), ('tablespace',), ('dimension',), ('dimension_slice',), ('chunk',), ('chunk_constraint',), ('chunk_index',), ('chunk_data_node',), ('metadata',), ('continuous_agg',), ('continuous_aggs_invalidation_threshold',), ('continuous_aggs_hypertable_invalidation_log',), ('continuous_aggs_materialization_invalidation_log',), ('hypertable_compression',), ('compression_algorithm',), ('compression_chunk_size',), ('remote_txn',), ('bgw_job',), ('bgw_job_stat',), ('bgw_policy_chunk_stats',), ('cache_inval_hypertable',), ('cache_inval_bgw_job',), ('cache_inval_extension',), ('django_migrations',), ('django_content_type',), ('api_businessinformation',), ('api_professionalexperience',), ('api_personallabequipments',), ('api_personaleducation',), ('api_personaldocumentupload',), ('api_personalcertificates',), ('api_languagesspoken',), ('api_inviteduser',), ('api_honorsandawards',), ('api_businesslabequipments',), ('api_user_role',), ('api_userrole',), ('django_

In [24]:
# print('challenge_creator_challengestatement:')
# print('-'*50)
# cursor.execute("Select * FROM challenge_creator_challengestatement LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_user:')
# print('-'*50)
# cursor.execute("Select * FROM api_user LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_personalinformation:')
# print('-'*50)
# cursor.execute("Select * FROM api_personalinformation LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_personallabequipments:')
# print('-'*50)
# cursor.execute("Select * FROM api_personallabequipments LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_businessinformation:')
# print('-'*50)
# cursor.execute("Select * FROM api_businessinformation LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_businesslabequipments:')
# print('-'*50)
# cursor.execute("Select * FROM api_businesslabequipments LIMIT 0")
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('api_businesslocation:')
# print('-'*50)
# cursor.execute("Select * FROM api_businesslocation LIMIT 0") 
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

# print('='*50)

# print('challenge_creator_comment:')
# print('-'*50)
# cursor.execute("Select * FROM challenge_creator_comment LIMIT 0") 
# colnames = [desc[0] for desc in cursor.description]
# print(colnames)

challenge_creator_challengestatement:
--------------------------------------------------
['id', 'challenge_title', 'challenge_description', 'challenge_location', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 'created_at', 'updated_at', 'company_name_id', 'user_id']
api_user:
--------------------------------------------------
['id', 'password', 'last_login', 'uid', 'email', 'username', 'can_invite_others', 'is_fresh_login', 'date_joined', 'is_active', 'is_admin', 'is_challenge_creator', 'is_solution_provider', 'is_manager', 'is_staff', 'created_at', 'modified_at', 'is_email_verified']
api_personalinformation:
--------------------------------------------------
['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 'city', 'state', 'country', 'created_at', 'modified_at', 'user_id', 'zip_code', 'profile_picture']
api_personallabequipments:
--------------

---

## Recommendations

In [6]:
def update_rcmd():
    
    try:
        collection = Collection(name = TABLE_NAME)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    skills = FieldSchema(name = field_name, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, skills], description = "example collection")
    collection = Collection(name = TABLE_NAME, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name, index_params=index_param)
    
#   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'skills', 'status_type', 
#    'post_type', 'is_active', 'is_archieve', 'created_at', 'updated_at', 'company_name_id', 'user_id']   
    
    cursor.execute("Select id, skills from challenge_creator_challengestatement")
    rows_pos = cursor.fetchall()
    
    lst_pid, lst_skills = [], []
    for i, j in rows_pos:
        lst_pid.append(i)
        lst_skills.append(j)
    
    sentence_embeddings = model.encode(lst_skills)
    sentence_embeddings = normalize(sentence_embeddings)
    print(type(sentence_embeddings))
    
    em = list(sentence_embeddings)
    mr = collection.insert([lst_pid, em])

In [7]:
# update_rcmd()

In [11]:
def query_rcmd(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    collection = Collection(TABLE_NAME)
    collection.load()
    results = collection.search(query_embeddings, field_name, param=search_params, limit=5, expr=None)
    
    recommendation = []
    for result in results[0]:
        
#       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'skills', 'status_type', 
#        'post_type', 'is_active', 'is_archieve', 'created_at', 'updated_at', 'company_name_id', 'user_id']        
        
        sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        get_user_name = "select username from api_user where id =" + str(rows[0][12]) + ";"
        cursor.execute(get_user_name)
        fetch_user_name = cursor.fetchall()
        fetch_user_name = fetch_user_name[0][0]
        
        if rows[0][11] is not None:
            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][11]) + ";"
            cursor.execute(get_user_name)
            fetch_company_name = cursor.fetchall()
            fetch_company_name = fetch_company_name[0][0]
        else:
            fetch_company_name = None
        
        if len(rows):
            recommendation.append({'id' : rows[0][0], 
                                   'challenge_title': rows[0][1],
                                   'challenge_description': rows[0][2],
                                   'challenge_location': rows[0][3],
                                   'skills': rows[0][4],
                                   'status_type': rows[0][5],
                                   'post_type': rows[0][6],
                                   'is_active': rows[0][7],
                                   'is_archieve': rows[0][8],
                                   'created_at': rows[0][9],
                                   'updated_at': rows[0][10],
                                   'company_name_id': rows[0][11],
                                   'user_id': rows[0][12],
                                   'user_name': fetch_user_name,
                                   'company_name': fetch_company_name})
    
    return recommendation

In [12]:
# rcmd = query_rcmd('ML')
# for i, j in enumerate(rcmd):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)

#     if i == 1:
#         break

0 -->
id : 1
challenge_title : How should I convert text of people's occupations into numbers for a machine learning project?
challenge_description : I'm predicting whether people will show up to their scheduled medical appointments or not. Should I vectorize people's occupations or would this cause the machine to learn useless patterns? Or should
challenge_location : Canada
skills : AI/ ML
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-10-04 11:26:38+00:00
updated_at : 2022-10-03 11:26:38+00:00
company_name_id : 1
user_id : 3
user_name : roshnikale1605
company_name : roshnikale1605
1 -->
id : 8
challenge_title : dfsds
challenge_description : dsghfkg
challenge_location : Seoul
skills : AI/ ML
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-10-11 06:52:12+00:00
updated_at : 2022-10-06 06:52:12+00:00
company_name_id : None
user_id : 3
user_name : roshnikale1605
company_name : None


---

## Search

In [13]:
def update_search():
    
    try:
        collection = Collection(name = TABLE_NAME_2)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    embds = FieldSchema(name = field_name_2, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, embds], description = "This table contails embeddings \
                              of mixed fields i.e. id, challenge_title, challenge_description, \
                              challenge_location and skills")
    collection = Collection(name = TABLE_NAME_2, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_2, index_params=index_param)
    
#   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'skills', 'status_type', 
#    'post_type', 'is_active', 'is_archieve', 'created_at', 'updated_at', 'company_name_id', 'user_id']   
    
    cursor.execute("Select id, challenge_title, skills, challenge_location, \
                    challenge_description from challenge_creator_challengestatement")
    rows_pos = cursor.fetchall()
    
    lst_pid, lst_mix = [], []
    for i in rows_pos:
        lst_pid.append(i[0])
        lst_mix.append(i[1] + ' ' + i[2] + ' ' + i[3] + ' ' + i[4])
    
    sentence_embeddings = model.encode(lst_mix)
    sentence_embeddings = normalize(sentence_embeddings)
    print(type(sentence_embeddings))
    
    em = list(sentence_embeddings)
    mr = collection.insert([lst_pid, em])

In [14]:
# update_search()

In [15]:
def query_search(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    collection = Collection(TABLE_NAME_2)
    collection.load()
    results = collection.search(query_embeddings, field_name_2, param=search_params, limit=5, expr=None)
    
    search = []
    for result in results[0]:
        
#       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'skills', 'status_type', 
#        'post_type', 'is_active', 'is_archieve', 'created_at', 'updated_at', 'company_name_id', 'user_id']
        
        sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        get_user_name = "select username from api_user where id =" + str(rows[0][12]) + ";"
        cursor.execute(get_user_name)
        fetch_user_name = cursor.fetchall()
        fetch_user_name = fetch_user_name[0][0]
        
        if rows[0][11] is not None:
            get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][11]) + ";"
            cursor.execute(get_user_name)
            fetch_company_name = cursor.fetchall()
            fetch_company_name = fetch_company_name[0][0]
        else:
            fetch_company_name = None
        
        if len(rows):
            search.append({'id' : rows[0][0], 
                           'challenge_title': rows[0][1],
                           'challenge_description': rows[0][2],
                           'challenge_location': rows[0][3],
                           'skills': rows[0][4],
                           'status_type': rows[0][5],
                           'post_type': rows[0][6],
                           'is_active': rows[0][7],
                           'is_archieve': rows[0][8],
                           'created_at': rows[0][9],
                           'updated_at': rows[0][10],
                           'company_name_id': rows[0][11],
                           'user_id': rows[0][12],
                           'user_name': fetch_user_name,
                           'company_name': fetch_company_name})
    
    return search

In [16]:
# search = query_search('meetings')
# for i, j in enumerate(search):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
#     if i == 1:
#         break

0 -->
id : 1
challenge_title : How should I convert text of people's occupations into numbers for a machine learning project?
challenge_description : I'm predicting whether people will show up to their scheduled medical appointments or not. Should I vectorize people's occupations or would this cause the machine to learn useless patterns? Or should
challenge_location : Canada
skills : AI/ ML
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-10-04 11:26:38+00:00
updated_at : 2022-10-03 11:26:38+00:00
company_name_id : 1
user_id : 3
user_name : roshnikale1605
company_name : roshnikale1605
1 -->
id : 8
challenge_title : dfsds
challenge_description : dsghfkg
challenge_location : Seoul
skills : AI/ ML
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-10-11 06:52:12+00:00
updated_at : 2022-10-06 06:52:12+00:00
company_name_id : None
user_id : 3
user_name : roshnikale1605
company_name : None


---

## Business

In [17]:
def update_biz():
    
    try:
        collection = Collection(name = TABLE_NAME_3)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    biz = FieldSchema(name = field_name_3, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, biz], description = "example collection")
    collection = Collection(name = TABLE_NAME_3, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_3, index_params=index_param)
    
#   ['id', 'userid', 'company_name', 'company_website', 'company_description', 
#    'company_email', 'company_phone', 'company_classification', 'invite_through_company', 
#    'created_at', 'modified_at', 'user_id']

#   ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
#    'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']
    
    cursor.execute("Select id, company_name, company_description, company_classification, \
                    company_website, company_email, company_phone from api_businessinformation")
    rows_pos = cursor.fetchall()
    
    lst_pid, lst_biz = [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])
        
        s = ''
        for m in i[1:]:

            if m is not None:
                s = s + ' ' + str(m)
        
        cursor.execute("Select company_address_line_1, company_address_line_2, company_city, \
                        company_state, company_country, zip_code from api_businesslocation \
                        WHERE business_id = {};".format(i[0]))
        rows_loc = cursor.fetchall()
        
        for j in rows_loc:
            for n in j:
                if n is not None:
                    s = s + ' ' + str(n)
        
        lst_biz.append(s)
    
    sentence_embeddings = model.encode(lst_biz)
    sentence_embeddings = normalize(sentence_embeddings)
    print(type(sentence_embeddings))
    
    em = list(sentence_embeddings)
    mr = collection.insert([lst_pid, em])

In [18]:
# update_biz()

In [19]:
def query_biz(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    collection = Collection(TABLE_NAME_3)
    collection.load()
    results = collection.search(query_embeddings, field_name_3, param=search_params, limit=5, expr=None)
    
    biz = []
    for result in results[0]:
        
#       ['id', 'userid', 'company_name', 'company_website', 'company_description', 
#        'company_email', 'company_phone', 'company_classification', 'invite_through_company', 
#        'created_at', 'modified_at', 'user_id']

#       ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
#        'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']
        
        sql = "select * from api_businessinformation where id = " + str(result.id) + ";"
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        sql = "select * from api_businesslocation where business_id = " + str(result.id) + ";"
        cursor.execute(sql)
        rows_loc = cursor.fetchall()
        
        loc = []
        for i in rows_loc:
            loc.append({'id': i[0],
                        'company_address_line_1': i[1],
                        'company_address_line_2': i[2],
                        'company_city': i[3],
                        'company_state': i[4],
                        'company_country': i[5],
                        'zip_code': i[6],
                        'created_at': i[7],
                        'modified_at': i[8],
                        'business_id': i[9],
                        'user_id': i[10]})
        
        if len(rows):
            biz.append({'id' : rows[0][0], 
                        'userid': rows[0][1],
                        'company_name': rows[0][2],
                        'company_website': rows[0][3],
                        'company_description': rows[0][4],
                        'company_email': rows[0][5],
                        'company_phone': rows[0][6],
                        'company_classification': rows[0][7],
                        'invite_through_company': rows[0][8],
                        'created_at': rows[0][9],
                        'modified_at': rows[0][10],
                        'user_id': rows[0][11],
                        'locations': loc})
    
    return biz

In [20]:
# biz = query_biz('India')
# for i, j in enumerate(biz):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
#     if i == 1:
#         break

0 -->
id : 1
userid : 2
company_name : VS Industries
company_website : None
company_description : None
company_email : vsindus@gmail.com
company_phone : 9983479374
company_classification : S23
invite_through_company : True
created_at : 2022-09-30 06:24:13.731000+00:00
modified_at : 2022-09-30 06:24:13.731000+00:00
user_id : 2
locations : [{'id': 1, 'company_address_line_1': '28/11 Baker Street', 'company_address_line_2': 'John Lane Avenue', 'company_city': 'Vadodra12', 'company_state': 'Gujarat', 'company_country': 'India', 'zip_code': 1, 'created_at': datetime.datetime(2022, 9, 30, 7, 37, 53, 783000, tzinfo=datetime.timezone.utc), 'modified_at': datetime.datetime(2022, 9, 30, 7, 40, 33, 446000, tzinfo=datetime.timezone.utc), 'business_id': 2, 'user_id': None}, {'id': 2, 'company_address_line_1': '28/11 Baker Street', 'company_address_line_2': 'John Lane Avenue', 'company_city': 'Vadodra', 'company_state': 'Gujarat', 'company_country': 'India', 'zip_code': 1, 'created_at': datetime.dat

---

## People

In [45]:
def update_ppl():
    
    try:
        collection = Collection(name = TABLE_NAME_4)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    ppl = FieldSchema(name = field_name_4, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, ppl], description = "example collection")
    collection = Collection(name = TABLE_NAME_4, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_4, index_params=index_param)
    
#   ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 
#    'headline', 'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 
#    'city', 'state', 'country', 'zip_code', 'created_at', 'modified_at', 'user_id']
    
    cursor.execute("Select id, first_name, last_name, personal_skills, job_title, \
                    headline, bio, personal_phone, address_line_1, address_line_2, \
                    city, state, country, personal_email, office_phone, zip_code from api_personalinformation")
    rows_pos = cursor.fetchall()
    
    lst_pid, lst_ppl = [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])
        
        s = ''
        for m in i[1:]:

            if m is not None:
                s = s + ' ' + str(m)
        
        lst_ppl.append(s)
    
    sentence_embeddings = model.encode(lst_ppl)
    sentence_embeddings = normalize(sentence_embeddings)
    print(type(sentence_embeddings))
    
    em = list(sentence_embeddings)
    mr = collection.insert([lst_pid, em])

In [46]:
# update_ppl()

<class 'numpy.ndarray'>


In [47]:
def query_ppl(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    collection = Collection(TABLE_NAME_4)
    collection.load()
    results = collection.search(query_embeddings, field_name_4, param=search_params, limit=5, expr=None)
    
    ppl = []
    for result in results[0]:   
        
#       ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 
#        'headline', 'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 
#        'city', 'state', 'country', 'zip_code', 'created_at', 'modified_at', 'user_id']
        
        sql = "select * from api_personalinformation where id = " + str(result.id) + ";"
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        if len(rows):
            ppl.append({'id' : rows[0][0], 
                        'first_name': rows[0][1],
                        'last_name': rows[0][2],
                        'personal_email': rows[0][3],
                        'personal_skills': rows[0][4],
                        'job_title': rows[0][5],
                        'headline': rows[0][6],
                        'bio': rows[0][7],
                        'office_phone': rows[0][8],
                        'personal_phone': rows[0][9],
                        'address_line_1': rows[0][10],
                        'address_line_2': rows[0][11],
                        'city': rows[0][12], 
                        'state': rows[0][13], 
                        'country': rows[0][14], 
                        'zip_code': rows[0][15], 
                        'created_at': rows[0][16], 
                        'modified_at': rows[0][17], 
                        'user_id': rows[0][18]})
    
    return ppl

In [25]:
# ppl = query_ppl('Dakota')
# for i, j in enumerate(ppl):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
#     if i == 1:
#         break

0 -->
id : 2
first_name : Manish
last_name : Shaw
personal_email : manishshaw12@gmail.com
personal_skills : Java
job_title : None
headline : None
bio : None
office_phone : 9997836452
personal_phone : None
address_line_1 : 5/4 Baker Street
address_line_2 : North Dakota
city : Dakota
state : Thames
country : Uganda
zip_code : 2022-09-30 06:23:15+00:00
created_at : 2022-10-21 14:36:18.232568+00:00
modified_at : 2
user_id : None


## Comment

In [48]:
def update_cmt():
    
    try:
        collection = Collection(name = TABLE_NAME_5)
        collection.drop()
    except:
        pass
    
    pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
    oid = FieldSchema(name = "oid", dtype = DataType.INT64)
    ppl = FieldSchema(name = field_name_5, dtype = DataType.FLOAT_VECTOR, dim = 384)
    schema = CollectionSchema(fields = [pid, oid, ppl], description = "Comment embeddings with its object_id and id")
    collection = Collection(name = TABLE_NAME_5, schema = schema)
    
    index_param = {
            "metric_type":"L2",
            "index_type":"IVF_SQ8",
            "params":{"nlist":1024}
        }
    collection.create_index(field_name=field_name_5, index_params=index_param)
    
#   ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 
#    'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']
    
    cursor.execute("Select id, object_id, user_comment from challenge_creator_comment")
    rows_pos = cursor.fetchall()
    
    lst_pid, lst_oid, lst_cmt = [], [], []
    for i in rows_pos:
        
        lst_pid.append(i[0])  
        lst_oid.append(i[1])
        lst_cmt.append(i[2])
    
    sentence_embeddings = model.encode(lst_cmt)
    sentence_embeddings = normalize(sentence_embeddings)
    print(type(sentence_embeddings))
    
    em = list(sentence_embeddings)
    mr = collection.insert([lst_pid, lst_oid, em])

In [49]:
# update_cmt()

<class 'numpy.ndarray'>


In [95]:
def query_cmt(q = ''):
    
    if type(q) != str:
        raise Exception("q must be a string")
 
    search_skills = q
    
    embed = model.encode(search_skills)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()
    
    collection = Collection(TABLE_NAME_5)
    collection.load()
    results = collection.search(query_embeddings, field_name_5, param=search_params, limit=5, expr=None)
    
    cmt = []
    for result in results[0]:   
        
#       ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 
#        'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']

        cmt_sql = "select * from challenge_creator_comment where id = " + str(result.id) + ";"
        cursor.execute(cmt_sql)
        rows_cmt = cursor.fetchall()

        sql = "select * from challenge_creator_challengestatement where id = {};".format(rows_cmt[0][1])
        cursor.execute(sql)
        rows = cursor.fetchall()
        
        if len(rows):
            cmt.append({'id' : rows[0][0], 
                                   'challenge_title': rows[0][1],
                                   'challenge_description': rows[0][2],
                                   'challenge_location': rows[0][3],
                                   'skills': rows[0][4],
                                   'status_type': rows[0][5],
                                   'post_type': rows[0][6],
                                   'is_active': rows[0][7],
                                   'is_archieve': rows[0][8],
                                   'created_at': rows[0][9],
                                   'updated_at': rows[0][10],
                                   'company_name_id': rows[0][11],
                                   'user_id': rows[0][12],
                                   'comment_id': rows_cmt[0][0],
                                   'user_comment': rows_cmt[0][2]})
    
    return cmt

In [96]:
# cmt = query_cmt('Dakota')
# for i, j in enumerate(cmt):
    
#     print(i, "-->")
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
    
#     if i == 1:
#         break

0 -->
id : 17
challenge_title : fgvhg
challenge_description : hjgfhjg
challenge_location : Afghanistan
skills : fghzfdxg,rxyhtr
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-11-02 06:02:03.272220+00:00
updated_at : 2022-11-02 06:02:03.272228+00:00
company_name_id : None
user_id : 52
comment_id : 33
user_comment : hi
1 -->
id : 10
challenge_title : Zippia has compiled a list of the top 20 tech skills (based on an analysis of over 3 million ).
challenge_description : Zippia has compiled a list of the top 20 tech skills (based on an analysis of over 3 million ).Zippia has compiled a list of the top 20 tech skills (based on an analysis of over 3 million ).Zippia has compiled a list of the top 20 tech skills (based on an analysis of over 3 million ).
challenge_location : India
skills : react-js,tailwind css,java,HTMl,css
status_type : New
post_type : Active
is_active : True
is_archieve : False
created_at : 2022-10-21 16:16:09.699063+00:00
update

---

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/updt', methods=['GET'])
def updt():
    
    update_rcmd()
    print('RCMD UPDATED')
    
    update_search()
    print('SEARCH UPDATED')
    
    update_biz()
    print('BIZ UPDATED')
    
    update_ppl()
    print('PPL UPDATED')
    
    update_cmt()
    print('CMT UPDATED')
    
    return ('', 204)

#===========================================#
    
@app.route('/get_data', methods=['GET'])
@cross_origin()
def get_data():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    rcmd = query_rcmd(q)
    search = query_search(q)
    biz = query_biz(q)
    ppl = query_ppl(q)
    cmt = query_cmt(q)
    
    return {'Recommendations': rcmd, 'Search': search, 'Business': biz, 'People': ppl, 'Comment': cmt}
    
@app.route('/get_rcmd', methods=['GET'])
@cross_origin()
def get_rcmd():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    rcmd = query_rcmd(q)
    
    return rcmd

In [2]:
if __name__ == '__main__':
    
    port = 80
    app.run(host='0.0.0.0', port=port, debug=True) # ssl_context=('cert.pem', 'key.pem')

---

In [1]:
get_ipython().system('jupyter nbconvert text_search_engine_alberta.ipynb --to python')

[NbConvertApp] Converting notebook text_search_engine_alberta.ipynb to python
[NbConvertApp] Writing 26150 bytes to text_search_engine_alberta.py
